In [1]:
import jax
import jax.numpy as jnp
from flax.struct import dataclass 

# Inter Robot

In [2]:
x = jnp.array([
    [1,1],
    [2,2],
    [3,3],
    [4,4]
])
states = jnp.stack((x, x + 0.1, x + 0.3))
states, states.shape

(Array([[[1. , 1. ],
         [2. , 2. ],
         [3. , 3. ],
         [4. , 4. ]],
 
        [[1.1, 1.1],
         [2.1, 2.1],
         [3.1, 3.1],
         [4.1, 4.1]],
 
        [[1.3, 1.3],
         [2.3, 2.3],
         [3.3, 3.3],
         [4.3, 4.3]]], dtype=float32, weak_type=True),
 (3, 4, 2))

In [6]:
@dataclass
class InterRobotMapping:
    points: jnp.ndarray
    other_robot: int

In [10]:
def find_closest_robot(pts: jnp.ndarray):
    def find_closest_pt_across_horizon(pt, other_pts):
        jax.debug.print("other pts: {}", other_pts)
        closest_index = jnp.argmin(jnp.linalg.norm(pt - other_pts, axis=1))
        return InterRobotMapping(other_pts[closest_index], closest_index)

    def find_batched_closest_pt(pts, i):
        other_pts = jnp.delete(pts, jnp.array([i]), assume_unique_indices=True, axis=0) # (N - 1, 4, 2)
        return jax.vmap(find_closest_pt_across_horizon, in_axes=(0, 1))(pts[i], other_pts)
    return jax.vmap(find_batched_closest_pt, in_axes=(None, 0))(pts, jnp.arange(pts.shape[0]))
closest_robots = find_closest_robot(states[:,:,0:2]) # ideally, should be # (N - 1, 4, 2)
closest_robots

other pts: [[1.1 1.1]
 [1.3 1.3]]
other pts: [[1.  1. ]
 [1.3 1.3]]
other pts: [[1.  1. ]
 [1.1 1.1]]
other pts: [[2.1 2.1]
 [2.3 2.3]]
other pts: [[2.  2. ]
 [2.3 2.3]]
other pts: [[2.  2. ]
 [2.1 2.1]]
other pts: [[3.1 3.1]
 [3.3 3.3]]
other pts: [[3.  3. ]
 [3.3 3.3]]
other pts: [[3.  3. ]
 [3.1 3.1]]
other pts: [[4.1 4.1]
 [4.3 4.3]]
other pts: [[4.  4. ]
 [4.3 4.3]]
other pts: [[4.  4. ]
 [4.1 4.1]]


InterRobotMapping(points=Array([[[1.1, 1.1],
        [2.1, 2.1],
        [3.1, 3.1],
        [4.1, 4.1]],

       [[1. , 1. ],
        [2. , 2. ],
        [3. , 3. ],
        [4. , 4. ]],

       [[1.1, 1.1],
        [2.1, 2.1],
        [3.1, 3.1],
        [4.1, 4.1]]], dtype=float32, weak_type=True), other_robot=Array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [1, 1, 1, 1]], dtype=int32))

# Obstacle

In [5]:
obstacle = jnp.array([
    [4.2, 4.2],
    [1.0, 1.0]
])

def find_closest_obstacle(states, obstacles):
    def find_closest_obstacle_for_pt(state_t, obstacles):
        closest_obstacle_idx = jnp.argmin(jnp.linalg.norm(state_t - obstacles, axis=1))
        return obstacles[closest_obstacle_idx]
    def batch_find_closest_obstacle(agent_states, obstacles):
        return jax.vmap(find_closest_obstacle_for_pt, in_axes=(0, None))(agent_states, obstacles)
    return jax.vmap(batch_find_closest_obstacle, in_axes=(0, None))(states, obstacles)
closest_obstacle = find_closest_obstacle(states, obstacle)
closest_obstacle, closest_obstacle.shape

(Array([[[1. , 1. ],
         [1. , 1. ],
         [4.2, 4.2],
         [4.2, 4.2]],
 
        [[1. , 1. ],
         [1. , 1. ],
         [4.2, 4.2],
         [4.2, 4.2]],
 
        [[1. , 1. ],
         [1. , 1. ],
         [4.2, 4.2],
         [4.2, 4.2]]], dtype=float32),
 (3, 4, 2))